In [ ]:
### imports ###
%load_ext autoreload
%autoreload 2

import numpy as np
import scipy as sc

import torch
import torch.nn as nn
from torch.nn.parameter import Parameter
import torch.nn.functional as F
import torch.optim as optim


import pickle


# add paths to access shared code
import sys
sys.path.append("..")
sys.path.append("../scripts/")

# import library implementing models
import neuroprob as nprb
from neuroprob import utils

# import utility code for model building/training/loading
import lib
import HDC

# get GPU device if available
gpu_dev = 0
dev = utils.pytorch.get_device(gpu=gpu_dev)

# use custom plotting settings
import matplotlib.pyplot as plt

#plt.style.use(['paper.mplstyle'])

# useful colormap
white = '#ffffff'
red = '#ff0000'
blue = '#0000ff'
weight_map = utils.plot.make_cmap([blue, white, red], 'weight_map')

Load model and dataset

In [18]:
checkpoint_dir = '../example/'
session_id = '12-120806'
phase = 'wake'

single_spikes = False
dt = 0.001

ll_mode = 'Uqd3'  # stands for universal count model with exponential-quadratic expansion and C = 3
filt_mode = ''
map_mode = 'svgp64'  # a sparse variational GP mapping with 64 inducing points
x_mode = 'hd-w-s-pos-t'  # observed covariates (behaviour)
z_mode = ''  # latent covariates
hist_len = 0
folds = 5  # cross-validation folds
delays = [0]
model_info = (ll_mode, filt_mode, map_mode, x_mode, z_mode, hist_len, folds, delays)

bin_size = 50  # ms
cv_run = -1  # test set is last 1/5 of dataset time series
delay = 0
batch_size = 5000  # size of time segments of each batch in dataset below

    
# data
dataset_tuple = HDC.get_dataset(session_id, phase, bin_size, single_spikes, path='../checkpoint/')

# model and train/test split based on which validation fold is given by cv_run
modelfit, fit_set, validation_set = lib.models.load_model(
    checkpoint_dir, model_info, dataset_tuple, HDC.inputs_used, HDC.enc_used, 
    delay, cv_run, batch_size, gpu_dev, tensor_type=torch.float, jitter=1e-5
)
rcov, units_used, tbin, resamples, rc_t, max_count, bin_size, metainfo, data_name = dataset_tuple

FileNotFoundError: [Errno 2] No such file or directory: '../example/HDC12-120806wake_Uqd3_H0_svgp64_X[hd-w-s-pos-t]_Z[]_50K37_0d0_5f-1'

In [ ]:
hd_sweep = torch.linspace(0, 2*np.pi, 100)

covariates = torch.cat([
    hd_sweep[:, None],  # sweep over head direction
    *[rcov[k].mean()*torch.ones((100, 1)) for k in range(1, len(rcov))], 
    # fill other behavioural covariates at value 0
], dim=-1)[None, None, ...]  # (tr, neurons, steps, covariate dims)

with torch.no_grad():
    P_mc = lib.helper.compute_P(
        modelfit, covariates, list(range(units_used)), MC=30, trials=1)  # predictive posterior
P_mc = P_mc.cpu()  # count probabilities of shape (MC, neurons, steps, count)